# MINITEST 3
## 3 vehicles in a row

Test setup: Honda(Civic) followed by Toyota (RAV4) followed by Honda (Pilot).
Note: The Toyota csv file have timestamps that are delayed by an hour due to internet issue in the pi.


In [43]:
import cantools
import csv
from strym import strymread
import strym
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
import pandas as pd
from strym import strymmap
from IPython.display import Video
import plotly.express as px
import plotly.graph_objects as go
toyota_dbcfile='/home/safwan/Documents/strym/strym/dbc/toyota_rav4_2019.dbc'
honda_dbcfile='/home/safwan/Documents/strym/strym/dbc/honda_pilot_2017.dbc'
folder = '/home/safwan/Documents/sfwithVM/cyverse/minitest3/2020_10_13/'

### Setting files paths 

In [44]:
honda_can=folder + '2020-10-13-12-18-59_5FNYF6H05HB089022_CAN_Messages.csv'
honda_gps= folder + '2020-10-13-12-18-59_5FNYF6H05HB089022_GPS_Messages.csv'

toyota_can=folder + '2020-10-13-13-19-13_2T3MWRFVXLW056972_CAN_Messages.csv'
toyota_gps= folder + '2020-10-13-13-19-13_2T3MWRFVXLW056972_GPS_Messages.csv'

CANcoach_can= folder + '2020-10-13-13-20-53-ID2.csv'

### Read the CSV file using `strymread`

In [ ]:
r_toyota = strymread(csvfile=CANcoach_can, dbcfile=toyota_dbcfile)
r_honda = strymread(csvfile=honda_can, dbcfile=honda_dbcfile)

### Plotting the map of the drive using `strymmap`

To be able to generate a map of a dirve please follow the instruction in https://jmscslgroup.github.io/strym/Strymmap_Example.html#Using-Google-Map

In [ ]:
api_key = input("Enter API Key: ")

In [ ]:
!echo "export MAP_BOX_API={api_key}" >> ~/.env

In [ ]:
strym.config["mapheight"] = 700
strym.config["mapwidth"] = 1250
strym.config["mapzoom"] = 12.20

In [ ]:
# g_toyota = strymmap(toyota_gps)
g_honda = strymmap(honda_gps)

In [ ]:
fig = g_honda.plotroute(interactive=True)

### Speed messages: 

In [ ]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
strym.config['interactive']=True
toyota_speed = r_toyota.get_ts(180,'SPEED') #  Tue Oct 13 11:20:54 AM MST 2020
toyota_speed['Time'] = toyota_speed['Time'] + 3600.0 ## adding ab hour to match the Honda timestamps
toyota_speed['Message'] = toyota_speed['Message'] * 0.277778 # change to m/s
strymread.plt_ts(toyota_speed, title= 'RAV4 speed msg')

honda_speed = r_honda.get_ts(msg = "WHEEL_SPEEDS", signal="WHEEL_SPEED_FL") #  Tue Oct 13 01:26:53 PM MST 2020
honda_speed=honda_speed [(honda_speed.Time >= 1602613254.839061 ) & (honda_speed.Time <= 1602620286.803147)] # trimming off the beginning so both files start at the same time
honda_speed['Message'] = honda_speed['Message'] * 0.277778 # change to m/s
strymread.plt_ts(honda_speed, title= 'Pilot Speed msg ')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=toyota_speed.Time, y=toyota_speed.Message,
                    mode='lines',
                    name='Toyota'))
fig.add_trace(go.Scatter(x=honda_speed.Time, y=honda_speed.Message,
                    mode='lines',
                    name='Honda'))
fig.update_layout(
    title="Speed",
    xaxis_title="Time",
    yaxis_title="m/s",
    legend_title="Legend Title",
)
fig.show()

### Lead distance from RAV4's lead message and radar traces

Directly from the lead message:

In [ ]:
lead_dist = r_toyota.get_ts(869,'LEAD_DISTANCE')
lead_dist=lead_dist[(lead_dist.Message < 252)]
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=lead_dist.Time, y=lead_dist.Message,
                    mode='markers',
                    name='Honda',
    ))
# fig1.add_trace(go.Scatter(x=toyota_speed.Time, y=toyota_speed.Message,
#                     mode='lines',
#                     name='Toyota'))
fig1.update_layout(
    title="RAV4 Lead distance",
    xaxis_title="Time",
    yaxis_title="m",
    legend_title="Legend Title",
)
fig1.show()

We can also get lead distance from the radar traces. However, the results still require further filtering to get clean data.

In [ ]:
toyota_long_dist = r_toyota.long_dist(np.arange(0, 16))
toyota_lat_dist = r_toyota.lat_dist(np.arange(0, 16))
toyota_rel = r_toyota.rel_velocity(np.arange(0, 16))

# Concatenate long, lat and relative vel of all tracks to a single dataframe
toyota_long_dist = pd.concat(toyota_long_dist)
toyota_lat_dist = pd.concat(toyota_lat_dist)
toyota_rel = pd.concat(toyota_rel)

toyota_long_dist['Long'] = toyota_long_dist['Message']
toyota_long_dist['Lat'] = toyota_lat_dist['Message']
toyota_long_dist['Relvel'] = toyota_rel['Message']
toyota_long_dist.drop(columns=['Message'], inplace=True)

toyota_lead_state = toyota_long_dist
toyota_lead_state

Filter out the toyota_lead_state so that it only contains tracks found by RADAR directly in front of the Toyota. For that we hypothesize that anything with lateral distance less than equal ± 0.5 is seen as a car driving in front of the Toyota

In [ ]:
toyota_lead_state = toyota_lead_state[np.abs(toyota_lead_state['Lat']) <= 0.5]
toyota_lead_state

Overlaid plots: lead distance and radar traces

In [ ]:
toyota_lead_state = toyota_lead_state[np.abs(toyota_lead_state['Long']) < 252]

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=toyota_lead_state.Time, y=toyota_lead_state.Long,
                    mode='markers',
                    name='RAV4 radar traces',
                    marker_color="MAGENTA",
#                           marker=dict(
#                             size=4,
#                           )
    ))
fig2.add_trace(go.Scatter(x=lead_dist.Time, y=lead_dist.Message,
                    mode='markers',
                    name='RAV4 lead dist',
                           marker_color="OLIVE",
#                           marker=dict(
#                             size=4,
#                           )
    ))                    
fig2.update_layout(
    title="RAV4 radar and lead",
    xaxis_title="Time",
    yaxis_title="m",
    legend_title="Legend Title",
)
fig2.show()
toyota_lead_state

From the above plot, we can see that futher filtering of the radar traces would gives a similar readings to the lead distance message.

Now, Resampling RAV4's speed data, and relative velocity data obtained from radar trace so that we can estimate speed of the lead vehicle (Honda Civic) in front of RAV4 by adding these two timeseries

In [ ]:
lead_rel = pd.DataFrame()
lead_rel['Time'] = toyota_lead_state_l['Time']
lead_rel['Message'] = toyota_lead_state_l['Relvel']

In [ ]:

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=lead_rel.Time, y=lead_rel.Message,
                    mode='markers',
                    name='RAV4 Relative velocity',
                    marker_color="MAGENTA",
                          marker=dict(
                            size=2,
                          )
    ))
                    
fig3.update_layout(
    title="RAV4 relative velocity when Honda civic is leading",
    xaxis_title="Time",
    yaxis_title="m/s",
    legend_title="Legend Title",
)
fig3.show()